In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm
import warnings
import statsmodels.api as sm
import os
warnings.filterwarnings("ignore")
os.chdir("../../db")
print(os.getcwd())

/Users/test/risksoft/TS/db


In [13]:
pb2024

,da,yield,year,pe,pb,fs_q,ticker
0,2024-01-02,3.07,111,21.92,2.32,112/3,3008
1,2024-01-03,3.18,111,21.13,2.24,112/3,3008
2,2024-01-04,3.23,111,20.85,2.21,112/3,3008
3,2024-01-05,3.29,111,20.42,2.16,112/3,3008
4,2024-01-08,3.31,111,20.30,2.15,112/3,3008
...,...,...,...,...,...,...,...
14,2024-06-24,0.00,111,0.0,0.73,113/1,4414
15,2024-06-25,0.00,111,0.0,0.72,113/1,4414
16,2024-06-26,0.00,111,0.0,0.72,113/1,4414
17,2024-06-27,0.00,111,0.0,0.70,113/1,4414


In [40]:
pb = pb2024.pivot(index='da', values="pe", columns='ticker')
pb = pb.loc['2024-01-01':'2024-12-31']
pb = pb.apply(lambda x: (x.astype(str).str.replace(",", '')))
pb = pb.astype(float)

In [41]:
pb2024 = pd.read_parquet("tw/pb_ratio/2024.parquet")
pb2024.replace("-", 0.0, inplace=True)
def from_pb_to_pdata(target='pb'):
    pb = pb2024.pivot(index='da', values="pe", columns='ticker')
    pb = pb.loc['2024-01-01':'2024-12-31']
    pb = pb.apply(lambda x: (x.astype(str).str.replace(",", '')))
    pb = pb.astype(float)
    pb.to_parquet(f'tw/pdata/{target}.parquet')
    return None

from_pb_to_pdata(target='pb')
from_pb_to_pdata(target='pe')


In [6]:
pct_close = pd.read_parquet("tw/pdata/close_pct.parquet")
pct_close_w = pct_close.resample("W-FRI").sum()

indexes = pct_close_w.index
tickers = pct_close_w.iloc[:, :].columns
indices = indexes[52:104]
forecast = np.zeros((len(indices), len(tickers)))
print(indices[0], indices[-1])
for index, date in enumerate(tqdm(indices, desc=f"arma model for top {len(tickers)}, enumerate by da")):
    pct_train = pct_close_w.loc[indexes[index]:date]
    for i in range(len(tickers)):
        series = pct_train[tickers[i]]
        if series.isna().any():
            prediction = 0
            print(tickers[i], "failed at", date)
            continue
            ###### ARIMA ma
        else:
            model = sm.tsa.arima.ARIMA(series, order=(2, 0, 0))
            model_fit = model.fit()
            prediction = model_fit.forecast().values[0]
        forecast[index, i] = prediction
factor = pd.DataFrame(forecast, columns=tickers, index=indices)


In [8]:
import requests
import json
stockNo = "2330"
da = 2024
url_pe = f"https://www.twse.com.tw/rwd/zh/block/BFIAUU_sd?stockNo={stockNo}&date={da}&response=json&_=1720554887120"

res = requests.get(url_pe)
js = res.json()
js

{'stat': 'OK',
 'date': 2024,
 'title': '2330 個股單一證券鉅額交易日成交資訊',
 'hints': '單位：元、股',
 'fields': ['日期', '交易別', '成交股數', '成交金額', '最高價', '最低價', '加權平均價'],
 'data': [['2024/01/02',
   '配對交易',
   '1,547,000',
   '916,343,204',
   '593.00',
   '587.95',
   '592.34'],
  ['2024/01/03',
   '配對交易',
   '1,437,000',
   '833,178,730',
   '580.04',
   '577.81',
   '579.80'],
  ['2024/01/04',
   '配對交易',
   '2,498,000',
   '1,447,858,433',
   '583.39',
   '579.19',
   '579.61'],
  ['2024/01/05',
   '配對交易',
   '3,085,000',
   '1,777,910,846',
   '578.00',
   '566.26',
   '576.31'],
  ['2024/01/08',
   '配對交易',
   '916,000',
   '533,997,730',
   '584.38',
   '582.50',
   '582.97'],
  ['2024/01/09',
   '配對交易',
   '465,000',
   '272,284,180',
   '585.96',
   '585.50',
   '585.56'],
  ['2024/01/10',
   '配對交易',
   '234,000',
   '136,610,480',
   '584.50',
   '583.50',
   '583.81'],
  ['2024/01/11',
   '配對交易',
   '3,255,000',
   '1,906,766,151',
   '587.50',
   '584.50',
   '585.80'],
  ['2024/01/12',
   '配對交易',